In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('2022Games.csv')

print(df)

##Linear

In [ ]:
y = df['HOME_TEAM_WINS']
print(y)

In [ ]:
Z = df.drop(['PTS_home','SEASON','TEAM_ID_home','HOME_TEAM_WINS','PTS_away','TEAM_ID_away'], axis =1)
Z['GAME_DATE_EST'] = 100* (Z['FG_PCT_home'] - Z['FG_PCT_away'])
Z['GAME_ID'] = 100*(Z['FT_PCT_home'] - Z['FT_PCT_away'])
Z['GAME_STATUS_TEXT'] = 100*(Z['FG3_PCT_home'] - Z['FG3_PCT_away'])
Z['HOME_TEAM_ID'] = Z['AST_home'] - Z['AST_away']
Z['VISITOR_TEAM_ID'] = Z['REB_home'] - Z['REB_away']
Z = Z.drop(['REB_away','AST_away','FG3_PCT_away','FT_PCT_away','FG_PCT_away','REB_home','AST_home','FG3_PCT_home','FT_PCT_home','FG_PCT_home'], axis =1)
Z

In [ ]:
from sklearn.model_selection import train_test_split

Z_train, Z_test, y_train, y_test = train_test_split(Z, y, test_size=0.2, random_state=100)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(Z_train, y_train)

In [ ]:
y_lr_train_pred = lr.predict(Z_train)
y_lr_test_pred = lr.predict(Z_test)

In [ ]:
y_test

In [ ]:
y_lr_test_pred

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
lr_train_mse = mean_squared_error(y_train,y_lr_train_pred)
lr_train_r2 = r2_score(y_train, y_lr_train_pred)

lr_test_mse = mean_squared_error(y_test, y_lr_test_pred)
lr_test_r2 = r2_score(y_test, y_lr_test_pred)

print("LR Test R2:", lr_test_r2)
print("LR Test MSE:", lr_test_mse)
print("LR Train R2:", lr_train_r2)
print("LR Test MSE:", lr_train_mse)

LR Test R2: 0.4692965344639455
LR Test MSE: 0.13211850034724948
LR Train R2: 0.5152044405632572
LR Test MSE: 0.11942646261105569


##Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [ ]:
y = df['HOME_TEAM_WINS']
print(y)

In [ ]:
Z = df.drop(['PTS_home','SEASON','TEAM_ID_home','HOME_TEAM_WINS','PTS_away','TEAM_ID_away'], axis =1)
Z['GAME_DATE_EST'] = 100* (Z['FG_PCT_home'] - Z['FG_PCT_away'])
Z['GAME_ID'] = 100*(Z['FT_PCT_home'] - Z['FT_PCT_away'])
Z['GAME_STATUS_TEXT'] = 100*(Z['FG3_PCT_home'] - Z['FG3_PCT_away'])
Z['HOME_TEAM_ID'] = Z['AST_home'] - Z['AST_away']
Z['VISITOR_TEAM_ID'] = Z['REB_home'] - Z['REB_away']
Z = Z.drop(['REB_away','AST_away','FG3_PCT_away','FT_PCT_away','FG_PCT_away','REB_home','AST_home','FG3_PCT_home','FT_PCT_home','FG_PCT_home'], axis =1)
Z

In [ ]:
print(f'Z : {Z.shape}')

In [ ]:
from sklearn.model_selection import train_test_split

Z_train, Z_test, y_train, y_test = train_test_split(Z, y, test_size=0.2, random_state=101)

### Hyperparameters

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [2,4]
min_samples_split = [2,5]
min_samples_leaf = [1,2]
bootstrap = [True, False]

In [ ]:
param_grid = {'n_estimators':n_estimators,
              'max_features':max_features,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf,
              'bootstrap':bootstrap
              }
print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [ ]:
rf_Model = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid,  cv = 3, verbose=2, n_jobs = 4)

In [ ]:
rf_Grid.fit(Z_train, y_train)

Fitting 3 folds for each of 320 candidates, totalling 960 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72,
                                          80]},
             verbose=2)

In [ ]:
rf_Grid.best_params_

{'bootstrap': False,
 'max_depth': 4,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 48}

In [ ]:
print(f'Train Accuracy - : {rf_Grid.score(Z_train, y_train):.3f}')
print(f'Test Accuracy - : {rf_Grid.score(Z_test, y_test):.3f}')

Train Accuracy - : 0.901
Test Accuracy - : 0.819


## Test on Past Years

In [ ]:
df2 = pd.read_csv('2021Games.csv')

print(df2)

In [ ]:
y2 = df2['HOME_TEAM_WINS']
print(y)

In [ ]:
Z2 = df2.drop(['PTS_home','SEASON','TEAM_ID_home','HOME_TEAM_WINS','PTS_away','TEAM_ID_away'], axis =1)
Z2['GAME_DATE_EST'] = 100* (Z2['FG_PCT_home'] - Z2['FG_PCT_away'])
Z2['GAME_ID'] = 100*(Z2['FT_PCT_home'] - Z2['FT_PCT_away'])
Z2['GAME_STATUS_TEXT'] = 100*(Z2['FG3_PCT_home'] - Z2['FG3_PCT_away'])
Z2['HOME_TEAM_ID'] = Z2['AST_home'] - Z2['AST_away']
Z2['VISITOR_TEAM_ID'] = Z2['REB_home'] - Z2['REB_away']
Z2 = Z2.drop(['REB_away','AST_away','FG3_PCT_away','FT_PCT_away','FG_PCT_away','REB_home','AST_home','FG3_PCT_home','FT_PCT_home','FG_PCT_home'], axis =1)
Z2

In [ ]:
from sklearn.model_selection import train_test_split

Z2_train, Z2_test, y2_train, y2_test = train_test_split(Z2, y2, test_size=0.2, random_state=101)

In [ ]:
print(f'Train Accuracy - : {rf_Grid.score(Z2_train, y2_train):.3f}')
print(f'Test Accuracy - : {rf_Grid.score(Z2_test, y2_test):.3f}')

Train Accuracy - : 0.831
Test Accuracy - : 0.838
